In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
data = pd.read_csv('C://Users//Zloy//mlcours//dats/telecom_churn.csv')

In [3]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [5]:
data['International plan'] = data['International plan'].map({'Yes':1, 'No':0})

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   int64  
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total day charge        3333 non-null   float64
 7   Total eve minutes       3333 non-null   float64
 8   Total eve calls         3333 non-null   int64  
 9   Total eve charge        3333 non-null   float64
 10  Total night minutes     3333 non-null   float64
 11  Total night calls       3333 non-null   int64  
 12  Total night charge      3333 non-null   float64
 13  Total intl minutes      3333 non-null   float64
 14  Total intl calls        3333 non-null   

In [8]:
y = data['Churn'].astype('int')

In [9]:
X = data.drop('Churn', axis=1)


In [10]:
X.shape, y.shape

((3333, 17), (3333,))

In [19]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=17)

In [15]:
X_train.shape, X_valid.shape

((2333, 17), (1000, 17))

In [16]:
first_tree = DecisionTreeClassifier(random_state=17)

In [23]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9138423504976518

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

0.8671274043984523

In [27]:
from sklearn.model_selection import GridSearchCV

In [29]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[0.5, 0.7, 1.0]}
tree_grid = GridSearchCV(first_tree, tree_params, cv=5)

In [30]:
%%time
tree_grid.fit(X_train, y_train);

CPU times: total: 1.33 s
Wall time: 1.35 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1.0]})

In [33]:
tree_grid.best_estimator_, tree_grid.best_score_

(DecisionTreeClassifier(max_depth=6, max_features=1.0, random_state=17),
 0.9417108564391468)

In [38]:
knn_params = {'n_neighbors': list(range(1, 20)) + list(range(50, 70))}
knn_grid = GridSearchCV(first_knn, knn_params, cv=50)

In [39]:
%%time
knn_grid.fit(X_train, y_train);

CPU times: total: 1min 4s
Wall time: 16.3 s


GridSearchCV(cv=50, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 50, 51, 52,
                                         53, 54, 55, 56, 57, 58, 59, 60, ...]})

In [41]:
knn_grid.best_params_, knn_grid.best_estimator_, knn_grid.best_score_

({'n_neighbors': 9}, KNeighborsClassifier(n_neighbors=9), 0.8744403330249768)

In [43]:
tree_valid_pred = tree_grid.predict(X_valid)

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(y_valid, tree_valid_pred)

0.946

In [47]:
1 - np.mean(y)

0.8550855085508551

In [48]:
from sklearn.tree import export_graphviz

In [50]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot', feature_names=X.columns, filled=True)

In [54]:
!ls -l *.png


"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [55]:
!dot-Tpng telecom_tree.dot -o telecom_tree.png

"dot-Tpng" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
